In [1]:
pip install pandas sqlalchemy psycopg2 pyarrow

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached SQLAlchemy-2.0.37-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached psycopg2-2.9.10-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached pyarrow-19.0.0-cp312-cp312-win_amd64.whl.metadata (3.4 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached greenlet-3.1.1-cp312-cp312-win_amd64.whl.metadata (3.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached SQLAlchemy-2.0.37-cp312-cp312-win_amd64.whl (2.1 MB)
Using cached psycopg2-2.9.10-cp312-cp312-win_amd64.whl (1.2 MB)
Using cached pyarrow-19.0.0-cp312-cp312-win_amd64.whl (25.2 MB)
Using cached greenlet-3.1.1-cp312-cp312-win_amd64.whl (299 kB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   -----------------------------

In [2]:
import argparse, os, sys
from time import time
import pandas as pd 
import pyarrow.parquet as pq
from sqlalchemy import create_engine

In [48]:
user = "root"
password = "root"
host = "localhost"
port = "5432"
db = "ny_taxi"
tb_main = "green_tripdata"
tb_lookup = "taxi_zone"
url_main = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz"
url_lookup = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"
save_path = "./data/"


In [64]:
file_name = url_main.rsplit('/', 1)[-1].strip()
full_path = os.path.join(save_path, file_name)
os.system(f"curl -L {url_main} -o {full_path}")

0

In [65]:
df = pd.read_csv(full_path,index_col=False)


C:\Users\topta\AppData\Local\Temp\ipykernel_25824\1440849558.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_path,index_col=False)


In [66]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2019-10-01 00:26:02,2019-10-01 00:39:58,N,1.0,112,196,1.0,5.88,18.00,0.50,0.5,0.00,0.00,NaN,0.3,19.30,2.0,1.0,0.0
1,1.0,2019-10-01 00:18:11,2019-10-01 00:22:38,N,1.0,43,263,1.0,0.80,5.00,3.25,0.5,0.00,0.00,NaN,0.3,9.05,2.0,1.0,0.0
2,1.0,2019-10-01 00:09:31,2019-10-01 00:24:47,N,1.0,255,228,2.0,7.50,21.50,0.50,0.5,0.00,0.00,NaN,0.3,22.80,2.0,1.0,0.0
3,1.0,2019-10-01 00:37:40,2019-10-01 00:41:49,N,1.0,181,181,1.0,0.90,5.50,0.50,0.5,0.00,0.00,NaN,0.3,6.80,2.0,1.0,0.0
4,2.0,2019-10-01 00:08:13,2019-10-01 00:17:56,N,1.0,97,188,1.0,2.52,10.00,0.50,0.5,2.26,0.00,NaN,0.3,13.56,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476381,NaN,2019-10-31 23:30:00,2019-11-01 00:00:00,NaN,NaN,65,102,NaN,7.04,29.57,2.75,0.5,0.00,0.00,NaN,0.0,32.82,NaN,NaN,NaN
476382,NaN,2019-10-31 23:03:00,2019-10-31 23:24:00,NaN,NaN,129,136,NaN,0.00,39.83,2.75,0.5,0.00,6.12,NaN,0.0,49.20,NaN,NaN,NaN
476383,NaN,2019-10-31 23:02:00,2019-10-31 23:23:00,NaN,NaN,61,222,NaN,3.90,23.11,2.75,0.5,0.00,0.00,NaN,0.0,26.36,NaN,NaN,NaN
476384,NaN,2019-10-31 23:42:00,2019-10-31 23:56:00,NaN,NaN,76,39,NaN,3.08,15.23,2.75,0.5,0.00,0.00,NaN,0.0,18.48,NaN,NaN,NaN


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       265 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


In [43]:
object_columns = df.select_dtypes(include=["object"]).columns

for col in object_columns:
    unique_types = df[col].apply(type).unique()
    print(f"Unique types in {col}: {unique_types}")


Unique types in Borough: [<class 'str'>]
Unique types in Zone: [<class 'str'> <class 'float'>]
Unique types in service_zone: [<class 'str'> <class 'float'>]


In [8]:
str_values = df[df["store_and_fwd_flag"].apply(lambda x: isinstance(x, str))]
str_values["store_and_fwd_flag"].unique()

array(['N', 'Y'], dtype=object)

In [9]:
float_values = df[df["store_and_fwd_flag"].apply(lambda x: isinstance(x, float))]
float_values['store_and_fwd_flag'].unique()

array([nan], dtype=object)

In [63]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')
df_iter = pd.read_csv(full_path,index_col=False,iterator=True, chunksize=100000)
with engine.connect() as connection:
    while True: 
        try:
            t_start = time()

            if file_name == "green_tripdata_2019-10.csv.gz":
                df = next(df_iter)
                df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])
                df["lpep_dropoff_datetime"] = pd.to_datetime(df["lpep_dropoff_datetime"])
                df.to_sql(name=tb_main, con=connection, if_exists='append')
            elif file_name == "taxi_zone_lookup.csv":
                df = next(df_iter)
                df.to_sql(name=tb_lookup, con=connection, if_exists='append')

            t_end = time()

            print('inserted another chunk, took %.3f second' % (t_end - t_start))

        except StopIteration:
            print("Finished ingesting data into the postgres database")
            break

inserted another chunk, took 0.109 second
Finished ingesting data into the postgres database
